# Logger

> A basic logging class


In [ ]:
# | default_exp client.Logger

In [ ]:
# | exporti
import datetime as dt

from typing import Optional, List
from dataclasses import dataclass, field

import traceback

from nbdev.showdoc import patch_to


In [ ]:
# | hide
from nbdev.showdoc import show_doc
from fastcore.test import test_eq

# Traceback


In [ ]:
# | export
@dataclass
class TracebackDetails:
    """result of _get_traceback_details function"""

    function_name: str
    file_name: str
    function_trail: str

    traceback_stack: [traceback.FrameSummary] = None
    parent_class: str = None

    def __init__(
        self,
        traceback_stack: [traceback.FrameSummary],
        parent_class=None,  # pass ParentClass.__name__
        debug_traceback: bool = False,
    ):
        self.function_trail = " -> ".join([line[2]
                                          for line in traceback_stack])

        self.function_name = traceback_stack[-1][2]
        self.file_name = traceback_stack[-1][0]
        self.parent_class = parent_class
        self.traceback_stack = traceback_stack


def get_traceback(
    root_module: str = "<module>",
    # drop entries from the top of stack to exclude the functions that retrieve the traceback
    num_stacks_to_drop=0,
    parent_class: str = None,
    debug_traceback: bool = False,
) -> TracebackDetails:  # returns a filtered list of FrameSummaries from traceback
    """method that retrieves traceback"""

    import traceback

    traceback_stack = traceback.extract_stack()

    # find the last module index
    module_index = 0

    for index, tb_line in enumerate(traceback_stack):
        function_name = tb_line[2]

        if function_name == root_module:
            module_index = index

    num_stacks_to_drop += 1  # adjust for init
    
    if module_index + num_stacks_to_drop >= len(traceback_stack)-1 :
        print("adjusting num_stacks_to_drop, consider revising `get_traceback` call")
        print({
            'stack_length': len(traceback_stack),
            'module_index': module_index,
            'num_stacks_to_drop_passed': num_stacks_to_drop
        })
        num_stacks_to_drop -= 1

    filtered_traceback_stack = traceback_stack[module_index:-
                                               num_stacks_to_drop]
    
    if debug_traceback:
        print({'len orig stack': len(traceback_stack),
            'len filtered stack': len(filtered_traceback_stack),
            'root_module_name': root_module, 'root_module_index': module_index,
            'stacks_to_drop': num_stacks_to_drop})

    return TracebackDetails(
        traceback_stack=filtered_traceback_stack,
        parent_class=parent_class,
        debug_traceback=debug_traceback,
    )


#### sample implementation of TracebackDetails


In [ ]:
class Foo:
    def __init__(self):
        pass

    def test_get_traceback_details(self, debug_traceback: bool = False):
        return get_traceback(parent_class=self.__class__.__name__,
                             debug_traceback=True)


# # print traceback details for test_get_details function
test_foo = Foo()

test_foo.test_get_traceback_details(
    debug_traceback=True
).__dict__


{'len orig stack': 24, 'len filtered stack': 2, 'root_module_name': '<module>', 'root_module_index': 21, 'stacks_to_drop': 1}


{'function_trail': '<module> -> test_get_traceback_details',
 'function_name': 'test_get_traceback_details',
 'file_name': '/tmp/ipykernel_27360/1437577808.py',
 'parent_class': 'Foo',
 'traceback_stack': [<FrameSummary file /tmp/ipykernel_27360/1437577808.py, line 13 in <module>>,
  <FrameSummary file /tmp/ipykernel_27360/1437577808.py, line 6 in test_get_traceback_details>]}

In [ ]:
# | export


class Logger:
    """log class with user customizeable output method"""

    root_module: str
    app_name: str

    logs: List[dict]
    breadcrumb: Optional[list]

    entity_id: Optional[str]
    domo_instance: Optional[str]
    # function to call with write_logs method.
    output_fn: Optional[callable] = None

    def __init__(
        self,
        app_name: str,  # name of the app for grouping logs
        root_module: Optional[str] = "<module>",  # root module for stack trace
        output_fn: Optional[
            callable
        ] = None,  # function to call with write_logs method.
        entity_id: Optional[str] = None,
        domo_instance: Optional[str] = None,
    ):
        self.app_name = app_name
        self.output_fn = output_fn
        self.root_module = root_module
        self.logs = []
        self.breadcrumb = []
        self.domo_instance = domo_instance
        self.entity_id = entity_id

    def _add_crumb(self, crumb):
        if crumb not in self.breadcrumb:
            self.breadcrumb.append(crumb)

    def _remove_crumb(self, crumb):
        if crumb in self.breadcrumb:
            self.breadcrumb.remove(crumb)

    def get_traceback(
        self,
        root_module: str = "<module>",
        # drop entries from the top of stack to exclude the functions that retrieve the traceback
        num_stacks_to_drop=0,
        parent_class: str = None,
    ):
        parent_class = parent_class or self.__class__.__name__

        num_stacks_to_drop += 1

        return get_traceback(
            root_module=root_module,
            num_stacks_to_drop=num_stacks_to_drop,
            parent_class=parent_class,
        )

In [ ]:
show_doc(Logger.get_traceback)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/client/Logger.py#L134){target="_blank" style="float:right; font-size:smaller"}

### Logger.get_traceback

>      Logger.get_traceback (root_module:str='<module>', num_stacks_to_drop=0,
>                            parent_class:str=None)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| root_module | str | <module> |  |
| num_stacks_to_drop | int | 0 | drop entries from the top of stack to exclude the functions that retrieve the traceback |
| parent_class | str | None |  |

### sample implementations of stack tracing methods


In [ ]:
# assert that the result of test_trace is of type FrameSummary
log = Logger(app_name="test traceback")


class Foo:
    logger: Logger

    def __init__(self):
        self.logger = Logger(app_name=self.__class__.__name__)

    def test_traceback(self):
        return self.logger.get_traceback().__dict__


test_foo = Foo()

test_foo.test_traceback()

{'function_trail': '<module> -> test_traceback',
 'function_name': 'test_traceback',
 'file_name': '/tmp/ipykernel_27360/2551220874.py',
 'parent_class': 'Logger',
 'traceback_stack': [<FrameSummary file /tmp/ipykernel_27360/2551220874.py, line 17 in <module>>,
  <FrameSummary file /tmp/ipykernel_27360/2551220874.py, line 12 in test_traceback>]}

## Logger logging methods


In [ ]:
# | export
@patch_to(Logger)
def _add_log(
    self: Logger,
    message: str,
    type_str: str,
    debug_log: bool = False,
    num_stacks_to_drop=3,
    entity_id: Optional[str] = None,
    domo_instance: Optional[str] = None,
) -> dict:
    """internal method to append message to log"""

    traceback_details = self.get_traceback(num_stacks_to_drop=num_stacks_to_drop)

    if debug_log:
        print(traceback_details.__dict__)

    new_row = {
        "date_time": dt.datetime.now(),
        "application": self.app_name,
        "log_type": type_str,
        "log_message": message,
        "breadcrumb": "->".join(self.breadcrumb),
        "domo_instance": domo_instance or self.domo_instance,
        "entity_id": entity_id or self.entity_id,
    }

    new_row.update(
        {
            "function_name": traceback_details.function_name,
            "file_name": traceback_details.file_name,
            "function_trail": traceback_details.function_trail,
        }
    )

    if debug_log:
        print(new_row)

    self.logs.append(new_row)

    return new_row


@patch_to(Logger)
def log_info(
    self: Logger,
    message,
    entity_id: Optional[str] = None,
    domo_instance: Optional[str] = None,
    debug_log=False,
    num_stacks_to_drop=3,
):
    """log an informational message"""
    return self._add_log(
        message=message,
        entity_id=entity_id,
        domo_instance=domo_instance,
        type_str="Info",
        num_stacks_to_drop=num_stacks_to_drop,
        debug_log=debug_log,
    )


@patch_to(Logger)
def log_error(
    self: Logger,
    message,
    entity_id: Optional[str] = None,
    domo_instance: Optional[str] = None,
    debug_log=False,
    num_stacks_to_drop=3,
):
    """log an error message"""

    return self._add_log(
        message=message,
        entity_id=entity_id,
        domo_instance=domo_instance,
        type_str="Error",
        num_stacks_to_drop=num_stacks_to_drop,
        debug_log=debug_log,
    )


@patch_to(Logger)
def log_warning(
    self: Logger,
    message,
    entity_id: Optional[str] = None,
    domo_instance: Optional[str] = None,
    debug_log=False,
    num_stacks_to_drop=3,
):
    """log a warning message"""

    return self._add_log(
        message=message,
        entity_id=entity_id,
        domo_instance=domo_instance,
        type_str="Warning",
        num_stacks_to_drop=num_stacks_to_drop,
        debug_log=debug_log,
    )

In [ ]:
logger = Logger(
    app_name="test",
)


def test_log():
    return logger.log_info("test the error returns type Info", debug_log=False)


test_log()

adjusting num_stacks_to_drop, consider revising `get_traceback` call
{'stack_length': 27, 'module_index': 21, 'num_stacks_to_drop_passed': 5}


{'date_time': datetime.datetime(2023, 9, 28, 13, 19, 58, 332482),
 'application': 'test',
 'log_type': 'Info',
 'log_message': 'test the error returns type Info',
 'breadcrumb': '',
 'domo_instance': None,
 'entity_id': None,
 'function_name': 'test_log',
 'file_name': '/tmp/ipykernel_27360/2082841627.py',
 'function_trail': '<module> -> test_log'}

## Outputting Logs

During Logger instantiation, users can pass a function, `output_fn` which will be called with the `Logger.output_log` method


In [ ]:
# | export


@patch_to(Logger)
def output_log(self: Logger):
    """calls the user defined output function"""
    return self.output_fn(self.logs)

##### Sample implementation with a custom write_logs method


In [ ]:
import pandas as pd


def custom_write_logs_fn(logs):
    print("printing logs")
    return pd.DataFrame(logs)


logger = Logger(app_name="test", output_fn=custom_write_logs_fn)


def test_error():
    try:
        if 1 == 1:
            raise Exception("random error")

    except Exception as e:
        logger.log_error(e)


def double_test():
    test_error()


# record first error
test_error()

# records second error nested inside double_test()
double_test()

logger.output_log()

adjusting num_stacks_to_drop, consider revising `get_traceback` call
{'stack_length': 27, 'module_index': 21, 'num_stacks_to_drop_passed': 5}
printing logs


,date_time,application,log_type,log_message,breadcrumb,domo_instance,entity_id,function_name,file_name,function_trail
0,2023-09-28 13:19:58.419760,test,Error,random error,,None,None,test_error,/tmp/ipykernel_27360/2322602224.py,<module> -> test_error
1,2023-09-28 13:19:58.420351,test,Error,random error,,None,None,double_test,/tmp/ipykernel_27360/2322602224.py,<module> -> double_test


In [ ]:
# | hide
# test that custom_write_logs_fn is stored in the logger as output_fn.
test_eq(logger.output_fn, custom_write_logs_fn)


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
